In [1]:
from modules.hand_visualizations import *
DATASET_PATH = "/Users/christina/.cache/kagglehub/datasets/drgfreeman/rockpaperscissors/versions/2"

In [ ]:
# Example: How to use different representations in training
if __name__ == "__main__":
    # Your dataset path
    dataset_path = "/Users/christina/.cache/kagglehub/datasets/drgfreeman/rockpaperscissors/versions/2"
    
    print("Exploring RPS dataset...")
    image_paths, labels, class_images = explore_rps_dataset(dataset_path)
    
    if image_paths:
        print(f"\nFound {len(image_paths)} images total")
        
        # Visualize samples from each class
        print("\nVisualizing sample images from each class...")
        visualize_multiple_samples(class_images, num_samples=3)
        
        # Process and visualize representations for one sample from each class
        print("\nProcessing hand representations...")
        for class_name, images in class_images.items():
            if images:
                print(f"\nProcessing {class_name} example...")
                sample_image = images[0]  # Take first image from each class
                representations = visualize_representations(sample_image)
                
                print(f"Representation shapes for {class_name}:")
                for key, value in representations.items():
                    if isinstance(value, np.ndarray):
                        print(f"  {key}: {value.shape}")
                
                break  # Just show one example for now
        
        # Create label mapping
        unique_labels = list(set(labels))
        label_to_idx = {label: idx for idx, label in enumerate(unique_labels)}
        idx_to_label = {idx: label for label, idx in label_to_idx.items()}
        
        print(f"\nLabel mapping: {label_to_idx}")
        
        # Convert string labels to integers
        numeric_labels = [label_to_idx[label] for label in labels]
        
        # Example: Create dataset for training
        print("\nCreating PyTorch datasets...")
        
        # Split data (simple example - you might want stratified split)
        from sklearn.model_selection import train_test_split
        
        train_paths, test_paths, train_labels, test_labels = train_test_split(
            image_paths, numeric_labels, test_size=0.2, random_state=42, stratify=numeric_labels
        )
        
        print(f"Train set: {len(train_paths)} images")
        print(f"Test set: {len(test_paths)} images")
        
        # Create datasets for different representations
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        
        # Example datasets
        datasets = {}
        for rep_type in ['raw_rgb', 'seg_mask', 'edges']:
            datasets[rep_type] = {
                'train': RPSRepresentationDataset(train_paths, train_labels, rep_type, transform),
                'test': RPSRepresentationDataset(test_paths, test_labels, rep_type, transform)
            }
        
        # Landmarks dataset (different transform)
        datasets['landmarks'] = {
            'train': RPSRepresentationDataset(train_paths, train_labels, 'landmarks', None),
            'test': RPSRepresentationDataset(test_paths, test_labels, 'landmarks', None)
        }
        
        print("\nDatasets created successfully!")
        print("Available representations:", list(datasets.keys()))
        
        # Test loading a sample
        print("\nTesting data loading...")
        for rep_type, dataset_dict in datasets.items():
            try:
                sample_data, sample_label = dataset_dict['train'][0]
                print(f"{rep_type}: data shape = {sample_data.shape if hasattr(sample_data, 'shape') else type(sample_data)}, label = {sample_label}")
            except Exception as e:
                print(f"Error loading {rep_type}: {e}")
    
    else:
        print("No images found. Please check the dataset path and structure.")

1. Raw RGB Representation